In [1]:
from tqdm import tqdm, trange
import json
from collections import namedtuple, defaultdict
import pandas as pd
import numpy as np
import torch
import os
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models.word2vec import Word2Vec
import random
from scipy.sparse import csr_matrix
import dgl

import pickle as pkl
import sys
sys.path.append('/mnt/nfs/zhangtl/utils')
from util import myout

Using backend: pytorch


## load

In [2]:
dataset = 'eng'
df = pd.read_csv(f'../../../01_process/data/{dataset}/Papers.csv') # 5353904
df.dropna(subset=['p_id', 'year'], inplace=True)
df.fillna({'refs': '', 'a_ids': '', 'v_id': '', 'fids': '', 'abst': ''}, inplace=True)
df

/home/ztl/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
0,3626509,BOUNDS ON ERROR PROBABILITY FOR DIRECT-SEQUENC...,2.744009e+09,1800.0,5,,"2632321535,3038285318,2293422917,2135355102",152076295.0,"10888,922,66978,633,7,66977,6359",
1,3499783,Fabrication of SUS 304 regularly cell-structur...,2.594487e+09,1800.0,0,,"271362782,1055509267,2477281293,2002957515",145550701.0,"45,751,41",
2,3571557,Queueing performance with impatient customers,2.622326e+09,1800.0,0,,"2533412808,2275493933,2177075905,2177075905,21...",184954342.0,"513,33,3208",
3,3221527,Solid-state synthesis of Mg-Si-Ge ternary ther...,2.397477e+09,1800.0,0,,"2002957515,2937526791,2940158953",145550701.0,"11102,45,1788,41,7575",
4,3537050,Solid-state synthesis of magnesium alloys.,2.608738e+09,1800.0,0,,2652625152,87940674.0,"41,45,1011,7575",
...,...,...,...,...,...,...,...,...,...,...
4616869,4599393,Vibration energy-harvesting using inerter-base...,3.037255e+09,2021.0,0,,"2751257219,2746920959,2080700420",128368299,"33,60375,16338,50117,272",
4616870,4600213,Vibration fatigue dynamic stress simulation un...,3.037409e+09,2021.0,0,"1570924281,1914113278,1967521159,1982760487,19...","2918373459,2898950381,2734374336",128368299,"3394,33,8384,9265,1123,2882,9063,12291,8571,27...",
4616871,4602078,Visible Light Communication for Position contr...,3.037753e+09,2021.0,0,,"3037865463,2925670350",5114360,"7,74,515,5124,111",
4616872,4210012,What attributes determine overall satisfaction...,2.954390e+09,2021.0,1,,"2774098955,2128152638,2941666933,2941752033",163152955,"39873,4610,5377,1160,4567,41,203,1685,60752,95...",


In [3]:
start_year, end_year = 2000, 2022
drop_id = df[(df.year < start_year) | (df.year >= end_year)].index
print(f'Drop {len(drop_id)} rows') # 559749
papers = df.drop(drop_id) # 3547589
papers

Drop 1145323 rows


,Unnamed: 0,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
1143223,1615279,"""A perfect and an absolute work"": Expertise, A...",2.049668e+09,2000.0,10,"570351917,636500714,646058792,1488345737,15281...",2483008919,145330833,"677,540,1216,7",
1143224,1642980,"""All That Is Solid Melts into Air"": Historians...",2.051454e+09,2000.0,20,"1482737005,2001555821,2007594931,2041656516,20...",2498865579,145330833,"26907,55837,1221,7,677",
1143225,1897373,"""Artificial lymphatic system"": a new approach ...",2.067766e+09,2000.0,25,"1495148118,1651729475,1751852575,1777491492,18...","2513036337,2552585347,2265020717,2143649098,22...",141152059,"5988,93,7370,18,7433,1064,1815,107070,1063,132...",
1143226,1234836,"""BUCKLE UP NOW!"" AN ENFORCEMENT PROGRAM TO ACH...",2.025348e+09,2000.0,36,"1989000751,1993415032,1994861034,2028247399,20...","2569654017,2116186335,2077360188,2004781743",84525459,"7,774,24,768,258,5761,25626,32052,5759,25533",
1143227,2648673,"""Burst"" technology with feedback-loop control ...",2.140178e+09,2000.0,15,"1981996417,2015483304,2022604818,2024323482,20...","2081067593,1910170253,2002783956,2124703562,22...",162355128,"6088,1627,2865,14778,1834,2647,906,67,1905,74,127",
...,...,...,...,...,...,...,...,...,...,...
4616869,4599393,Vibration energy-harvesting using inerter-base...,3.037255e+09,2021.0,0,,"2751257219,2746920959,2080700420",128368299,"33,60375,16338,50117,272",
4616870,4600213,Vibration fatigue dynamic stress simulation un...,3.037409e+09,2021.0,0,"1570924281,1914113278,1967521159,1982760487,19...","2918373459,2898950381,2734374336",128368299,"3394,33,8384,9265,1123,2882,9063,12291,8571,27...",
4616871,4602078,Visible Light Communication for Position contr...,3.037753e+09,2021.0,0,,"3037865463,2925670350",5114360,"7,74,515,5124,111",
4616872,4210012,What attributes determine overall satisfaction...,2.954390e+09,2021.0,1,,"2774098955,2128152638,2941666933,2941752033",163152955,"39873,4610,5377,1160,4567,41,203,1685,60752,95...",


## build graph

In [4]:
dataset = 'eng'
p_id2emb = pkl.load(open(f'../../../01_process/data_papers/{dataset}/p_id2emb.pkl', 'rb'))
myout(p_id2emb)
def update_idx(idx, dic, cnt, feats, rel, feat_dim, no_emb):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        if rel == 0:
            try:
                feats.append(torch.from_numpy(p_id2emb[idx]).to(torch.float32))
            except:
                feats.append(torch.rand(feat_dim).to(torch.float32)-1)
                no_emb += 1
        elif rel == 1:
            feats.append(torch.rand(feat_dim).to(torch.float32)+1)
        elif rel == 2:
            feats.append(torch.rand(feat_dim).to(torch.float32)+2)
        elif rel == 3:
            feats.append(torch.rand(feat_dim).to(torch.float32))
    return dic, cnt, feats, no_emb

p_id2emb : len=4610510, dict([2744008977: [-0.2011116   0.11407492 -0.01325371 -0.6488667   0.69704175 -0.1612054
  0.34205592  0.14228885  0.05002832 -0.06491772  0.30456957 -0.18895042
 -0.22660041 -0.03145821 -0.44711566  0.03577932 -0.05042038  0.64587665
 -0.30335286  0.0909399  -0.24160022 -0.3166463   0.02679984  0.02551357
 -0.5173049  -0.16615717 -0.12160152 -0.19455467 -0.02735693 -0.69780976
  0.06373935 -0.11394241 -0.3298608   0.31499207  0.5529367   0.35981727
 -0.22529428 -0.15911178  0.34598318  0.481173   -0.58751106  0.07201648
  0.20815559  0.12152148 -0.20890702  0.08048207 -0.11499951 -0.26069972
 -0.20925738 -0.00115123  0.10817701  0.15661149 -0.54238284  0.527184
 -0.29627788 -0.43041602 -0.46078777  0.27262685 -0.71294755  0.09387419
 -0.21954371 -0.23917158  0.4872084   0.07108764  0.15741368 -0.3493085
 -0.1483959   0.134062   -0.33791113  0.1256511   0.2930844  -0.24119379
  0.510157    0.10569976  0.16936302  0.5158686   0.5524114  -0.32154262
  0.0040607  

In [5]:
start_year, end_year = 2000, 2022
feat_dim = 128

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []
for i in trange(len(papers)):
    ts = int(papers['year'].iloc[i])
    p_id = int(papers['p_id'].iloc[i])
    refs = papers['refs'].iloc[i]
    
    id2nid, cnt, feats, no_emb = update_idx(p_id, id2nid, cnt, feats, 0, feat_dim, no_emb)
    if len(refs)>0:
        rlst = refs.split(',')
        for ref in rlst:
            if ref != '':
                ref = int(ref)
                id2nid, cnt, feats, no_emb = update_idx(ref, id2nid, cnt, feats, 0, feat_dim, no_emb)
                lst.append((id2nid[p_id], id2nid[ref], 0, ts))
        
feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
tsp = torch.tensor([item[3] for item in lst])
myout(feat, src, tgt, rel, tsp)

100%|██████████| 3467410/3467410 [07:30<00:00, 7699.21it/s]


feat : shape=torch.Size([16070002, 128])
tensor([[-0.1284, -0.2340,  0.2399,  ...,  0.3161, -0.1484,  0.0439],
        [-0.2736, -0.3476, -0.2481,  ..., -0.9190, -0.8620, -0.5330],
        [-0.1984, -0.2639, -0.0518,  ..., -0.4240, -0.1529, -0.6509],
        ...,
        [-0.2232, -0.0773, -0.9879,  ..., -0.2870, -0.7742, -0.1368],
        [-0.2546, -0.6168, -0.0654,  ..., -0.5158, -0.2794, -0.2683],
        [-0.6386, -0.1388, -0.6990,  ..., -0.0378, -0.5443, -0.5241]])
src : shape=torch.Size([70944260]), tensor([       0,        0,        0,  ..., 16069997, 16069997, 16069997])
tgt : shape=torch.Size([70944260]), tensor([       1,        2,        3,  ..., 14900531, 12823844, 16070001])
rel : shape=torch.Size([70944260]), tensor([0, 0, 0,  ..., 0, 0, 0])
tsp : shape=torch.Size([70944260]), tensor([2000, 2000, 2000,  ..., 2021, 2021, 2021])


In [6]:
graph = dgl.graph((src, tgt), num_nodes=len(feat))
graph.ndata['feat'] = feat

nid2id = {vv: kk for kk, vv in id2nid.items()}
graph.ndata['raw_nid'] = torch.arange(len(feat))

graph.edata['rel'] = rel
graph.edata['ts'] = tsp
graph

Graph(num_nodes=16070002, num_edges=70944260,
      ndata_schemes={'feat': Scheme(shape=(128,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64)})

In [7]:
dgl.save_graphs(f'../data/{dataset}/graph.bin', [graph])

In [8]:
json.dump(id2nid, open(f'../data/{dataset}/id2nid.json', 'w'))

## gen cites

In [9]:
cites = {}
print(start_year, end_year)
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

papers.fillna({'refs': ''}, inplace=True)
for i in trange(len(papers)):
    year = int(papers['year'].iloc[i])
    refs = papers['refs'].iloc[i]
    if len(refs)>0:
        rlst = refs.split(',')
        rlst = [int(item) for item in rlst]
        for ref in rlst:
            if ref != '':
                cites[year][ref] += 1

2000 2022


100%|██████████| 3467410/3467410 [01:57<00:00, 29573.04it/s]


In [10]:
tsp = graph.edata['ts']
ts_vals, ts_cuts = np.unique(tsp.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(tsp.numpy())]

num_ts = len(ts_vals)
ts_infos = np.stack([ts_vals, ts_cuts[0:num_ts], ts_cuts[1:num_ts+1]]).transpose()
myout(ts_cuts, ts_vals, ts_infos)

ts_cuts : len=23, list([0, 706610, 1474625, ..., 64805656, 70940561, 70944260])
ts_vals : shape=(22,), [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021]
ts_infos : shape=(22, 3)
[[    2000        0   706610]
 [    2001   706610  1474625]
 [    2002  1474625  2317970]
 [    2003  2317970  3276711]
 [    2004  3276711  4401863]
 [    2005  4401863  5743632]
 [    2006  5743632  7353546]
 [    2007  7353546  9264361]
 [    2008  9264361 11373763]
 [    2009 11373763 13782366]
 [    2010 13782366 16449590]
 [    2011 16449590 19549563]
 [    2012 19549563 23044832]
 [    2013 23044832 27200439]
 [    2014 27200439 31881933]
 [    2015 31881933 37226624]
 [    2016 37226624 43139973]
 [    2017 43139973 49896488]
 [    2018 49896488 56780371]
 [    2019 56780371 64805656]
 [    2020 64805656 70940561]
 [    2021 70940561 70944260]]


In [11]:
labels = {}
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = ts_infos[np.where(ts_infos[:, 0]==year)[0][0], 1:]
    nids = graph.edges()[0][left:right].unique().tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf
    

2020: 100%|██████████| 1/1 [00:00<00:00, 34.35it/s, pdf=173968, year=2020]
2021: 0it [00:00, ?it/s]


In [12]:
labels[2005]

,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,1607052370,146485,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1635531668,241325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2168382411,342046,2.0,2.0,0.0,0.0,1.0,3.0,3.0,1.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2136250291,494353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2078286011,504128,0.0,1.0,6.0,6.0,8.0,4.0,8.0,8.0,7.0,18.0,18.0,18.0,6.0,10.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79982,1520229545,2685699,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79983,2232695114,2685703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79984,1931399056,2685714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
79985,1482640779,2685717,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
pkl.dump(labels, open(f'../data/{dataset}/labels.pkl', 'wb'))

## cum log labels

In [14]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-2): # 2000, 2020
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[2005]

21


,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,1607052370,146485,0.693147,1.098612,1.386294,1.609438,1.945910,2.079442,2.197225,2.197225,2.197225,2.197225,2.302585,2.302585,2.397895,2.397895,2.397895,2.397895
1,1635531668,241325,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2168382411,342046,1.098612,1.609438,1.609438,1.609438,1.791759,2.197225,2.484907,2.564949,2.639057,2.772589,2.772589,2.772589,2.833213,2.833213,2.833213,2.833213
3,2136250291,494353,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
4,2078286011,504128,0.000000,0.693147,2.079442,2.639057,3.091043,3.258096,3.526361,3.737670,3.891820,4.204693,4.442651,4.634729,4.691348,4.779123,4.836282,4.836282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79982,1520229545,2685699,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
79983,2232695114,2685703,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
79984,1931399056,2685714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.693147,0.693147
79985,1482640779,2685717,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294


In [15]:
pkl.dump(labels_cum_log, open(f'../data/{dataset}/labels_cum_log.pkl', 'wb'))